In [ ]:
import yfinance as yf
import os
import pandas as pd
import pandas_datareader as data
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tickerSymbol='AAPL'

tickerData=yf.Ticker(tickerSymbol)

In [ ]:
df=tickerData.history(period='id',start='2010-01-01',end='2023-03-14')
df.head()

In [ ]:
df.tail()

In [ ]:
df=df.reset_index()
df.head()

In [ ]:
df=df.drop(['Date','Dividends','Stock Splits'],axis=1)
df.head()

In [ ]:
df=df.drop(['index'],axis=1)
df.head()

In [ ]:
plt.plot(df.Close)


In [ ]:
df

In [ ]:
ma100=df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize = (16,8))
plt.plot(df.Close)
plt.plot(ma100,'r')

In [ ]:
ma200=df.Close.rolling(200).mean()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df.Close)
plt.plot(ma100,'r')
plt.plot(ma200,'g')

In [ ]:
df.shape

In [ ]:
#Splitting Data into Training and Testing
data_training=pd.DataFrame(df['Close'][0:int(len(df)*0.70)])

In [ ]:
data_testing=pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])
print(data_training.shape)
print(data_testing.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array=scaler.fit_transform(data_training)
data_training_array

In [ ]:
#

In [ ]:
x_train=[]
y_train=[]

for i in range(100,data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)

x_train



In [ ]:
#ML Model

In [ ]:
import keras.layers

In [ ]:
pip install keras

In [ ]:
!pip uninstall keras-nightly
!pip uninstall -y tensorflow
!pip install keras==2.1.6
!pip install tensorflow==1.15.0
!pip install h5py==2.10.0

In [ ]:
from keras.layers import Dense,Dropout,LSTM

In [ ]:
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,
               input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
past_100_days=data_training.tail(100)

In [ ]:
final_df=past_100_days.append(data_testing,ignore_index=True)

In [ ]:
 x_test=[]
 y_test=[]


In [ ]:
input_data=scaler.fit_transform(final_df)

In [ ]:
input_data

In [ ]:
input_data.shape

In [ ]:
for i in range(100,input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i,0])

In [ ]:
x_test,y_test=np.array(x_test),np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
y_predicted=model.predict(x_test)
y_predicted.shape

In [ ]:
y_test

In [ ]:
scaler.scale_

In [ ]:
scale_factor=1/0.00928253
y_predicted=y_predicted*scale_factor
y_test=y_test*scale_factor

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test,'b',label='Original_Price' )
plt.plot(y_predicted,'r',label='Predicted_Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()